In [ ]:
import pyedflib
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy 
import pandas as pd
import polars as pl
from edf_file_reader import EdfFileReader

print (os.environ['CONDA_DEFAULT_ENV'])

In [ ]:
#set path to main directory
path = '../..'

#open the df with data labels
df = pd.read_excel(f"{path}/data/data_labels.xlsx")

In [ ]:
#for loop to open each bdf file and instanitate the EdfFileReader class,read the file, 
#then loop through each signal and
#save each signal as a seperate parquet file and save it in the directory of the file it came from


#identify the signals to extract
signal_ls = ['ECG_A', 'ECG_B', 'HR', 'RR_int', 'resp', 'skin_imp', 'RR', 'skin_temp', 'ACC_X', 'ACC_Y', 'ACC_Z', 'posture', 'posture_fine', 'activity', 'body_temp']

#loop throught the df
for index, row in df.iterrows():
    #get the file path for the biosensors
    biosensor_id = row['Biosensor ID']
    file_name = row['File name']
    print(biosensor_id)
    file_path = f"{path}/bdf_files/{file_name}/{biosensor_id}/{biosensor_id}.bdf" 
    
    #instantiate the EdfFileReader class
    file_reader = EdfFileReader(file_path)
    file_reader.read_file()
    
    #enter the file duration back into the original df (df)
    dur = file_reader.file_dur
    #df.at[index, 'file duration(s)'] = dur

    #loop through each signal and save it as a parquet file under the biosensor directory
    for signal in signal_ls:
        signal_df = pl.from_numpy(getattr(file_reader, signal), )
        file_path = f"{path}/bdf_files/{file_name}/{biosensor_id}/{signal}"
        signal_df.write_parquet(file_path + '.parquet')